### Neuroevolution:

- Allgemein:
    - Bei jeder Mutation gibt es eine mutation rate (wie viele Parameter werden verändert) und mutation strength (wie stark werden die ausgewählten parameter mutiert werden)
    - Mutationen basieren auf Gaussian noise
    - Die agents, die für die nächste Generation mutiert werden sollen, werden mithilfe von fitness proportional selection (roulette wheel selection) ausgewählt. Vorher wird die fitness allerdings normalisiert und softmax angewandt (Temperatur ist Hyperparameter)
    - Elitismus: Die besten n (Hyperparameter) Agents pro Generation werden unverändert direkt in die nächste Generation übernommen
    - Jede Generation hat exakt viele Agenten
    - Die nächste Generation basiert immer auf den Eliten + mutierten Nachkommen von Selektierten (Eliten machen bei der roulette wheel selection mit)
    - Agent bricht nach ersten Tod ab bzw nach max steps ab
    - Es wird der standard reward von super mario gym benutzt
    - Aktuell wird kein cross over verwendet

- Klassenaufteilung:

    - NeuroevolutionNet:
        - Konfigurierbares neuronales Netz bestehend aus CNN + MLP.
        - Flexible Layer-Definition via cnn_config / mlp_config
        - Gewichtsinitialisierung mit Xavier init
        - Mutiert mit zufälligen hinzufügen von gaussian noise

    - NeuroevolutionAgent:
        - Führt Preprocessing des Beobachtungsbildes durch (Graustufen, Resize, Normalisierung).
        - Wählt Aktion per argmax(logits) aus dem Netzoutput.
        - Bewertet Episode berechnet Gesamtfitness
        - Erkennt eigenen Tod (erster Tod ist permanent, nur 1 von 3 Leben wird genutzt)

    - NeuroevolutionTrainer:
        - Führt den gesamten evolutionären Trainingsprozess durch
        - Startpopulation erzeugen (Kopien des Basisnetzes).
        - Agenten evaluieren und selektieren (roulette wheel selection ).
        - Neue Generation erzeugen (inkl. Elitismus).
        - Speichert Fitnessmetriken (best/avg/min).
        - Zeichnet bestes Modell im Video auf
        - Plot der Fitnessverläufe (matplotlib), Speicherung als PNG.
        - Speichert bestes model und kann model laden


##### ToDo:
- Training pausieren und fortsetzen können (um zb eine gesamte erfolgreiche population mit einer extra angepassten decaykurve zu trainieren)

##### Nice to have:
- Verbesserung der GPU Auslastung (Hat sich als schwierig herausgestellt, im Moment ist cpu-only schneller!)
- Frame skipping um performance zu erhöhen (Auch schwierig hauptlast ist nicht die entscheidung des agenten sondern die laufende mario umgebung)
- Percentage clipping für die eliteagents?

## Ideen:
- Finden einer Lösung für alle Level
- Finden der schnellsten Lösung für ein Level
- Analysieren, was das cnn von (ausgewählten, wie zb die besten) agenten sieht
- Training mit allen agenten aus random_hyperparameter_search_1 als startagents (z.B. pop 100, gens 100, hohe temperatur, elitism 5)
- Training mit den 10 besten agenten aus r_h_s_1 (z.B. pop 100, gens 100, elitism 10)
- In sinnvollen Bereich gridsearch durchführen
- Manuelles auswerten der Videos und "gut aussehende" Agenten als Elite in ein neues Training übernehmen
- Analyse der rhs1 Daten
- Überprüfen ob Agent auch in anderen Leveln gut abschneidet
- Finden eines super-Agenten der so viele Level wie möglich schaffen kann

# Random Hyperparameter Search 1

Es werden 100 Runs mit zufälligen Hyperparametern gemacht

In [ ]:
import random
from src.non_reinforcement.neuroevolution.neuroevolution_trainer import NeuroevolutionTrainer
from src.non_reinforcement.neuroevolution.neuroevolution_net import NeuroevolutionNet
from gym_super_mario_bros.actions import RIGHT_ONLY
%config InlineBackend.figure_format = 'retina'

# Constants
CNN_CONFIG = [
    {"out_channels": 16, "kernel_size": 8, "stride": 4},
    {"out_channels": 32, "kernel_size": 4, "stride": 2},
]
MLP_CONFIG = [
    32 * 9 * 9,  # Must match flattened output of CNN
    256,
    128,
    len(RIGHT_ONLY)
]
ENV_NAME = 'SuperMarioBros-1-1-v0'
ACTION_SET = RIGHT_ONLY
DEVICE = "cpu"
DIRECTORY = "../../runs/neuroevolution/random_hyperparameter_search_1"
GENERATIONS = 20
POPULATION_SIZE = 50
MAX_STEPS_PER_EPISODE = 2500

# Hyperparameter search ranges
MUTATION_RATE_SELECTION_INTERVAL = (0.00, 0.3)

MUTATION_STRENGTH_SELECTION_INTERVAL = (0.00, 0.2)

ROULETTE_TEMP_MIN = 0.5
ROULETTE_TEMP_MAX = 2.0

ELITISM_OPTIONS = [0, 1, 2, 3, 4, 5]

for _ in range (100):
    model = NeuroevolutionNet(
        input_channels=1,
        num_actions=len(RIGHT_ONLY),
        cnn_config=CNN_CONFIG,
        mlp_config=MLP_CONFIG
    )

    r1 = round(random.uniform(*MUTATION_RATE_SELECTION_INTERVAL), 2)
    r2 = round(random.uniform(*MUTATION_RATE_SELECTION_INTERVAL), 2)
    mutation_rate_range = tuple(sorted((r1, r2)))

    r1 = round(random.uniform(*MUTATION_STRENGTH_SELECTION_INTERVAL), 2)
    r2 = round(random.uniform(*MUTATION_STRENGTH_SELECTION_INTERVAL), 2)
    mutation_strength_range = tuple(sorted((r1, r2)))

    roulette_wheel_selection_temperature = round(random.uniform(ROULETTE_TEMP_MIN, ROULETTE_TEMP_MAX), 2)
    elitism = random.choice(ELITISM_OPTIONS)

    trainer = NeuroevolutionTrainer(
        base_model=model,
        env_name=ENV_NAME,
        action_set=ACTION_SET,
        device=DEVICE,
        directory=DIRECTORY,
        generations=GENERATIONS,
        population_size=POPULATION_SIZE,
        max_steps_per_episode=MAX_STEPS_PER_EPISODE,
        mutation_rate_range=mutation_rate_range,
        mutation_strength_range=mutation_strength_range,
        roulette_wheel_selection_temperature=roulette_wheel_selection_temperature,
        elitism=elitism
    )
    trainer.run()
    trainer.record_best_agent()
    trainer.save_metrics()
    trainer.save_best_model()

# Exploitation test 1

Bester Agent aus random_hyperparameter_search_1 wird genommen und maximal exploited, falls besserer agent raus kommt, wiederhole mit diesen agenten <br>
Versuch (aus rhs_1) max_fit_2125__avg_fit_267__env_name_SuperMarioBros-1-1-v0__action_set_5 -> max_fit_2360__avg_fit_422__env_name_SuperMarioBros-1-1-v0__action_set_5 <br>
Versuch (aus et_1) max_fit_2360__avg_fit_422__env_name_SuperMarioBros-1-1-v0__action_set_5 -> max_fit_2360__avg_fit_447__env_name_SuperMarioBros-1-1-v0__action_set_5 (keine verbesserung) <br>
Versuch (aus et_1) max_fit_2360__avg_fit_447__env_name_SuperMarioBros-1-1-v0__action_set_5 -> max_fit_2749__avg_fit_609__env_name_SuperMarioBros-1-1-v0__action_set_5 <br>
Versuch (aus et_1) max_fit_2749__avg_fit_609__env_name_SuperMarioBros-1-1-v0__action_set_5 -> max_fit_3047__avg_fit_1060__env_name_SuperMarioBros-1-1-v0__action_set_5 (level geschafft)

In [ ]:
from src.non_reinforcement.neuroevolution.neuroevolution_trainer import NeuroevolutionTrainer
from src.non_reinforcement.neuroevolution.neuroevolution_net import NeuroevolutionNet
from gym_super_mario_bros.actions import RIGHT_ONLY
%config InlineBackend.figure_format = 'retina'

# Constants
CNN_CONFIG = [
    {"out_channels": 16, "kernel_size": 8, "stride": 4},
    {"out_channels": 32, "kernel_size": 4, "stride": 2},
]
MLP_CONFIG = [
    32 * 9 * 9,  # Must match flattened output of CNN
    256,
    128,
    len(RIGHT_ONLY)
]
MODEL = NeuroevolutionNet(
    input_channels=1,
    num_actions=len(RIGHT_ONLY),
    cnn_config=CNN_CONFIG,
    mlp_config=MLP_CONFIG
)
ENV_NAME = 'SuperMarioBros-1-1-v0'
ACTION_SET = RIGHT_ONLY
DEVICE = "cpu"
DIRECTORY = "../../runs/neuroevolution/exploitation_test_1"
GENERATIONS = 100
POPULATION_SIZE = 100
MAX_STEPS_PER_EPISODE = 2500
MUTATION_RATE_RANGE = (0.00, 0.02)
MUTATION_STRENGTH_RANGE = (0.00, 0.02)
ROULETTE_WHEEL_SELECTION_TEMPERATURE = 0.01
ELITISM = 1

trainer = NeuroevolutionTrainer(
        base_model=MODEL,
        env_name=ENV_NAME,
        action_set=ACTION_SET,
        device=DEVICE,
        directory=DIRECTORY,
        generations=GENERATIONS,
        population_size=POPULATION_SIZE,
        max_steps_per_episode=MAX_STEPS_PER_EPISODE,
        mutation_rate_range=MUTATION_RATE_RANGE,
        mutation_strength_range=MUTATION_STRENGTH_RANGE,
        roulette_wheel_selection_temperature=ROULETTE_WHEEL_SELECTION_TEMPERATURE,
        elitism=ELITISM
    )

trainer.load_model("../../runs/neuroevolution/exploitation_test_1/max_fit_2749__avg_fit_609__env_name_SuperMarioBros-1-1-v0__action_set_5/best_model_2749.0.pt")
trainer.run()
trainer.record_best_agent()
trainer.save_metrics()
trainer.save_best_model()

# Promising Hyperparameter Test 1
Nehme die Hyperparameter von einen Training mit guten max und avg fitness und wiederhole den test mit mehr generationen. Benutze Hyperparameter von (rht_1) max_fit_1863__avg_fit_292__env_name_SuperMarioBros-1-1-v0__action_set_5

1. Ergebnis: max_fit_2237__avg_fit_399__env_name_SuperMarioBros-1-1-v0__action_set_5
- ziemlich gute, verdächtig logarithmisch aussehende fitness kurve => es wird wahrscheinlich mutation rate / strength decay benötigt, für bessere ergebnisse

2. Ergebnis: max_fit_2352__avg_fit_369__env_name_SuperMarioBros-1-1-v0__action_set_5
- Wiederholung von 1. für Bestätigung.(wieder ungefähr dasselbe problem, bestätigt die Vermutung das man mutation decay braucht)


3. Ergebnis: max_fit_3048__avg_fit_718__env_name_SuperMarioBros-1-1-v0__action_set_5
- Wiederholung mit mutation decay (extreme Verbesserung, man sieht wann der decay stärker wird, avg fitness zum schluss bei 2000 anstatt bei 500 (!), es wurde eine lösung für das level gefunden)

In [ ]:
from src.non_reinforcement.neuroevolution.neuroevolution_trainer import NeuroevolutionTrainer
from src.non_reinforcement.neuroevolution.neuroevolution_net import NeuroevolutionNet
from gym_super_mario_bros.actions import RIGHT_ONLY
%config InlineBackend.figure_format = 'retina'

# Constants
CNN_CONFIG = [
    {"out_channels": 16, "kernel_size": 8, "stride": 4},
    {"out_channels": 32, "kernel_size": 4, "stride": 2},
]
MLP_CONFIG = [
    32 * 9 * 9,  # Must match flattened output of CNN
    256,
    128,
    len(RIGHT_ONLY)
]
MODEL = NeuroevolutionNet(
    input_channels=1,
    num_actions=len(RIGHT_ONLY),
    cnn_config=CNN_CONFIG,
    mlp_config=MLP_CONFIG
)
ENV_NAME = 'SuperMarioBros-1-1-v0'
ACTION_SET = RIGHT_ONLY
DEVICE = "cpu"
DIRECTORY = "../../runs/neuroevolution/promising_hyperparameter_test_1"
GENERATIONS = 200
POPULATION_SIZE = 50
MAX_STEPS_PER_EPISODE = 2500
MUTATION_RATE_RANGE = (0.14, 0.25)
MUTATION_STRENGTH_RANGE = (0.14, 0.19)
ROULETTE_WHEEL_SELECTION_TEMPERATURE = 0.98
ELITISM = 4
MUTATION_RATE_SIGMOID_DECAY = {"a": 0.05, "b": 0.058, "c": 100}
MUTATION_STRENGTH_SIGMOID_DECAY = {"a": 0.05, "b": 0.058, "c": 100}

trainer = NeuroevolutionTrainer(
        base_model=MODEL,
        env_name=ENV_NAME,
        action_set=ACTION_SET,
        device=DEVICE,
        directory=DIRECTORY,
        generations=GENERATIONS,
        population_size=POPULATION_SIZE,
        max_steps_per_episode=MAX_STEPS_PER_EPISODE,
        mutation_rate_range=MUTATION_RATE_RANGE,
        mutation_strength_range=MUTATION_STRENGTH_RANGE,
        roulette_wheel_selection_temperature=ROULETTE_WHEEL_SELECTION_TEMPERATURE,
        elitism=ELITISM,
        mutation_rate_sigmoid_decay=MUTATION_RATE_SIGMOID_DECAY,
        mutation_strength_sigmoid_decay=MUTATION_STRENGTH_SIGMOID_DECAY
    )

trainer.run()
trainer.record_best_agent()
trainer.save_metrics()
trainer.save_best_model()

# Individual Level Training 1 (Finden einer Lösung für 1-2 (keine Lösung gefunden))

Da der promising_hyperparameter_test_1/max_fit_3048__avg_fit_718__env_name_SuperMarioBros-1-1-v0__action_set_5 so ein großer Erfolg war, versuche ich jetzt basierend auf den hyperparametern eine Lösung für Level 1-2 zu finden.

1-2 erster Versuch: max_fit_2213__avg_fit_444__env_name_SuperMarioBros-1-2-v0__action_set_5
- deutlich schlechteres Ergebnis als bei 1-1

1-2 zweiter Versuch: max_fit_2017__avg_fit_437__env_name_SuperMarioBros-1-2-v0__action_set_5
- wieder ein schlechtes Ergebnis, obwohl die anzahl der Generationen von 200 auf 400 verdoppelt wurde, es scheint als ob vor allen dingen von einer deutlich niedrigeren mutationsrate/stärke profitiert wird und die phase mit hoher mutationsrate/stärke nur anfänglich etwas gebracht hat, bis so Generation 20?? Es gab nur leichte verbesserungen bis so gen 100 in der explorationsphase. Vielleicht sind 2,5k steps für dieses level einfach zu wenig für eine gute exploration/explotation?

1-2 dritter Versuch: max_fit_2042__avg_fit_423__env_name_SuperMarioBros-1-2-v0__action_set_5
- Relativ schlechtes Ergebnis, obwohl max steps von 2,5k auf 5k erhöht wurden, gens wurden (aus Zeitgründen) von 400 auf 200 verringert, also liegt es nicht daran, dass die agenten zu wenig Zeit haben?

1-2 vierter Versuch: max_fit_1850__avg_fit_632__env_name_SuperMarioBros-1-2-v0__action_set_5
- Relativ schlechte max fitness, dafür aber sehr gute steigung der avg fitness. max steps bleiben auf 5k, die decayfunktion wurde angepasst, sodass, der decay früher beginnt und es einen längeren übergang auf niedrigere decay werte gibt. Min decay von 5 prozent auf 1 prozent verringert um ein noch präziseres finetuning zu ermöglichen. Trotzdem werden die gens von 200 auf 300 angehoben, damit es eine noch längere explotationsphase/fintuningphase gibt

1-2 fünfter Versuch: max_fit_1863__avg_fit_1286__env_name_SuperMarioBros-1-2-v0__action_set_5
- Leider wieder ein schlechtes ergebnis, die avg fitness und min fitness ist zwar stark angestiegen, allerdings nur bis zur max fitness. Wahrscheinlich ist die Gesamtpopulation einfach zu einer minimalen Mutation der Eliten geworden

1-2 sechster Versuch: max_fit_2429__avg_fit_669__env_name_SuperMarioBros-1-2-v0__action_set_5
- Bisher bestes Ergebnis, da irgendwie garnichts funktioniert habe ich einfach den ersten versuch mit 5k max steps und 300 statt 200 generationen wiederholt. Bis gen 200 stieg die avg und max fitness relativ gut, aber ab 200 stagniert beides. Wahrscheinlich ist die decaykurve bis 200 perfekt, aber dannach braucht man eine decaykurve die langsam linear von zb 0.05 auf 0.01 fällt

1-2 siebter Versuch: max_fit_2441__avg_fit_531__env_name_SuperMarioBros-1-2-v0__action_set_5
- Da sechster Versuch der beste gewesen ist und ich jetzt keine Zeit habe ein feature zu entwickeln was es erlaubt das gesamte Training fortzuführen, nachdem es beendet wurde, versuche ich jetzt einfach mal was passiert wenn man den sechsten Versuch nimmt und die population von 50 auf 100 erhöht (außer verdopplung der Laufzeit von ca 1 tag auf 2 tage). Ergebnis: nur minimal besser als sechster versuch trotz doppelter population?

1-2 achter Versuch: max_fit_2193__avg_fit_721__env_name_SuperMarioBros-1-2-v0__action_set_5
- Ich versuche den Übergang zwischen hoch und tief der decayfunktion schneller zu machen, verringere aus Zeitgründen gens von 300 auf 200. Ergebnis mittelgut, die avg fitnesskurve sieht sehr verdächtig nach der simoiddecayfunktion aus

1-2 neunter Versuch: max_fit_2441__avg_fit_2154__env_name_SuperMarioBros-1-2-v0__action_set_5
- Da ich in moment für jeden run ca 1,5 Tage Laufzeit habe und noch fertig werden möchte, finetune ich jetzt einfach das beste model max_fit_2441__avg_fit_531__env_name_SuperMarioBros-1-2-v0__action_set_5, um überhaupt noch eine Lösung für das Level zu bekommen. Keine Verbesserung

1-2 zehnter Versuch: max_fit_2441__avg_fit_2125__env_name_SuperMarioBros-1-2-v0__action_set_5
- Benutze diesmal keine decay funktion, sondern benutze ähnliches setup wie in exploitation_test_1. Keine Verbesserung, vielleicht range verrringern?

1-2 elfter Versuch: max_fit_2441__avg_fit_2424__env_name_SuperMarioBros-1-2-v0__action_set_5
- Verringere mut range und strength. Keine Verbesserung

In [ ]:
from src.non_reinforcement.neuroevolution.neuroevolution_trainer import NeuroevolutionTrainer
from src.non_reinforcement.neuroevolution.neuroevolution_net import NeuroevolutionNet
from gym_super_mario_bros.actions import RIGHT_ONLY
%config InlineBackend.figure_format = 'retina'

# Constants
CNN_CONFIG = [
    {"out_channels": 16, "kernel_size": 8, "stride": 4},
    {"out_channels": 32, "kernel_size": 4, "stride": 2},
]
MLP_CONFIG = [
    32 * 9 * 9,  # Must match flattened output of CNN
    256,
    128,
    len(RIGHT_ONLY)
]
MODEL = NeuroevolutionNet(
    input_channels=1,
    num_actions=len(RIGHT_ONLY),
    cnn_config=CNN_CONFIG,
    mlp_config=MLP_CONFIG
)
ENV_NAME = 'SuperMarioBros-1-3-v0'
ACTION_SET = RIGHT_ONLY
DEVICE = "cpu"
DIRECTORY = "../../runs/neuroevolution/individual_level_training_1/1-3"
GENERATIONS = 200
POPULATION_SIZE = 50
MAX_STEPS_PER_EPISODE = 2500
MUTATION_RATE_RANGE = (0.14, 0.25)
MUTATION_STRENGTH_RANGE = (0.14, 0.19)
ROULETTE_WHEEL_SELECTION_TEMPERATURE = 0.98
ELITISM = 4
MUTATION_RATE_SIGMOID_DECAY = {"a": 0.05, "b": 0.058, "c": 100}
MUTATION_STRENGTH_SIGMOID_DECAY = {"a": 0.05, "b": 0.058, "c": 100}

trainer = NeuroevolutionTrainer(
        base_model=MODEL,
        env_name=ENV_NAME,
        action_set=ACTION_SET,
        device=DEVICE,
        directory=DIRECTORY,
        generations=GENERATIONS,
        population_size=POPULATION_SIZE,
        max_steps_per_episode=MAX_STEPS_PER_EPISODE,
        mutation_rate_range=MUTATION_RATE_RANGE,
        mutation_strength_range=MUTATION_STRENGTH_RANGE,
        roulette_wheel_selection_temperature=ROULETTE_WHEEL_SELECTION_TEMPERATURE,
        elitism=ELITISM,
        mutation_rate_sigmoid_decay=MUTATION_RATE_SIGMOID_DECAY,
        mutation_strength_sigmoid_decay=MUTATION_STRENGTH_SIGMOID_DECAY
    )

trainer.run()
trainer.record_best_agent()
trainer.save_metrics()
trainer.save_best_model()

# Individual Level Training 1 (Finden einer Lösung für 1-3) Keine Lösung gefunden

1-3 erster Versuch: max_fit_821__avg_fit_390__env_name_SuperMarioBros-1-3-v0__action_set_5
- Benutze dieselben Parameter wie 1-1/max_fit_3048__avg_fit_718__env_name_SuperMarioBros-1-1-v0__action_set_5. Ergebnis: Verdächtigerweise direkt ab der schwebenen platform keine Verbesserung mehr

1-3 zweiter Versuch: max_fit_716__avg_fit_492__env_name_SuperMarioBros-1-3-v0__action_set_5
- Benutze dieselben Parameter wie in 1-2/max_fit_2441__avg_fit_531__env_name_SuperMarioBros-1-2-v0__action_set_5, da hier der höchste score in level 1-2 erzielt wurde. Ergebnis noch schlechter als vorher, mario stürtzt sogar vor der schwebenen Platform ab

In [ ]:
from src.non_reinforcement.neuroevolution.neuroevolution_trainer import NeuroevolutionTrainer
from src.non_reinforcement.neuroevolution.neuroevolution_net import NeuroevolutionNet
from gym_super_mario_bros.actions import RIGHT_ONLY
%config InlineBackend.figure_format = 'retina'

# Constants
CNN_CONFIG = [
    {"out_channels": 16, "kernel_size": 8, "stride": 4},
    {"out_channels": 32, "kernel_size": 4, "stride": 2},
]
MLP_CONFIG = [
    32 * 9 * 9,  # Must match flattened output of CNN
    256,
    128,
    len(RIGHT_ONLY)
]
MODEL = NeuroevolutionNet(
    input_channels=1,
    num_actions=len(RIGHT_ONLY),
    cnn_config=CNN_CONFIG,
    mlp_config=MLP_CONFIG
)
ENV_NAME = 'SuperMarioBros-1-3-v0'
ACTION_SET = RIGHT_ONLY
DEVICE = "cpu"
DIRECTORY = "../../runs/neuroevolution/individual_level_training_1/1-3"
GENERATIONS = 300
POPULATION_SIZE = 100
MAX_STEPS_PER_EPISODE = 5000
MUTATION_RATE_RANGE = (0.14, 0.25)
MUTATION_STRENGTH_RANGE = (0.14, 0.19)
ROULETTE_WHEEL_SELECTION_TEMPERATURE = 0.98
ELITISM = 4
MUTATION_RATE_SIGMOID_DECAY = {"a": 0.05, "b": 0.058, "c": 100}
MUTATION_STRENGTH_SIGMOID_DECAY = {"a": 0.05, "b": 0.058, "c": 100}

trainer = NeuroevolutionTrainer(
        base_model=MODEL,
        env_name=ENV_NAME,
        action_set=ACTION_SET,
        device=DEVICE,
        directory=DIRECTORY,
        generations=GENERATIONS,
        population_size=POPULATION_SIZE,
        max_steps_per_episode=MAX_STEPS_PER_EPISODE,
        mutation_rate_range=MUTATION_RATE_RANGE,
        mutation_strength_range=MUTATION_STRENGTH_RANGE,
        roulette_wheel_selection_temperature=ROULETTE_WHEEL_SELECTION_TEMPERATURE,
        elitism=ELITISM,
        mutation_rate_sigmoid_decay=MUTATION_RATE_SIGMOID_DECAY,
        mutation_strength_sigmoid_decay=MUTATION_STRENGTH_SIGMOID_DECAY
    )

trainer.run()
trainer.record_best_agent()
trainer.save_metrics()
trainer.save_best_model()

# Individual Level Training 1 (Finden einer Lösung für 1-4) Lösung gefunden

Erster Versuch: max_fit_2180__avg_fit_1208__env_name_SuperMarioBros-1-4-v0__action_set_5
- Benutze dieselben Parameter wie in 1-2/max_fit_2441__avg_fit_531__env_name_SuperMarioBros-1-2-v0__action_set_5, da hier der höchste score in level 1-2 erzielt wurde. Level wurde relativ schnell geschafft

In [ ]:
from src.non_reinforcement.neuroevolution.neuroevolution_trainer import NeuroevolutionTrainer
from src.non_reinforcement.neuroevolution.neuroevolution_net import NeuroevolutionNet
from gym_super_mario_bros.actions import RIGHT_ONLY
%config InlineBackend.figure_format = 'retina'

# Constants
CNN_CONFIG = [
    {"out_channels": 16, "kernel_size": 8, "stride": 4},
    {"out_channels": 32, "kernel_size": 4, "stride": 2},
]
MLP_CONFIG = [
    32 * 9 * 9,  # Must match flattened output of CNN
    256,
    128,
    len(RIGHT_ONLY)
]
MODEL = NeuroevolutionNet(
    input_channels=1,
    num_actions=len(RIGHT_ONLY),
    cnn_config=CNN_CONFIG,
    mlp_config=MLP_CONFIG
)
ENV_NAME = 'SuperMarioBros-1-4-v0'
ACTION_SET = RIGHT_ONLY
DEVICE = "cpu"
DIRECTORY = "../../runs/neuroevolution/individual_level_training_1/1-4"
GENERATIONS = 300
POPULATION_SIZE = 100
MAX_STEPS_PER_EPISODE = 5000
MUTATION_RATE_RANGE = (0.14, 0.25)
MUTATION_STRENGTH_RANGE = (0.14, 0.19)
ROULETTE_WHEEL_SELECTION_TEMPERATURE = 0.98
ELITISM = 4
MUTATION_RATE_SIGMOID_DECAY = {"a": 0.05, "b": 0.058, "c": 100}
MUTATION_STRENGTH_SIGMOID_DECAY = {"a": 0.05, "b": 0.058, "c": 100}

trainer = NeuroevolutionTrainer(
        base_model=MODEL,
        env_name=ENV_NAME,
        action_set=ACTION_SET,
        device=DEVICE,
        directory=DIRECTORY,
        generations=GENERATIONS,
        population_size=POPULATION_SIZE,
        max_steps_per_episode=MAX_STEPS_PER_EPISODE,
        mutation_rate_range=MUTATION_RATE_RANGE,
        mutation_strength_range=MUTATION_STRENGTH_RANGE,
        roulette_wheel_selection_temperature=ROULETTE_WHEEL_SELECTION_TEMPERATURE,
        elitism=ELITISM,
        mutation_rate_sigmoid_decay=MUTATION_RATE_SIGMOID_DECAY,
        mutation_strength_sigmoid_decay=MUTATION_STRENGTH_SIGMOID_DECAY
    )

trainer.run()
trainer.record_best_agent()
trainer.save_metrics()
trainer.save_best_model()

# Individual Level Training 1 (Finden einer Lösung für 2-1)

Erster Versuch: max_fit_1827__avg_fit_515__env_name_SuperMarioBros-2-1-v0__action_set_5
- Benutze dieselben Parameter wie in 1-2/max_fit_2441__avg_fit_531__env_name_SuperMarioBros-1-2-v0__action_set_5, da hier der höchste score in level 1-2 erzielt wurde.

In [ ]:
from src.non_reinforcement.neuroevolution.neuroevolution_trainer import NeuroevolutionTrainer
from src.non_reinforcement.neuroevolution.neuroevolution_net import NeuroevolutionNet
from gym_super_mario_bros.actions import RIGHT_ONLY
%config InlineBackend.figure_format = 'retina'

# Constants
CNN_CONFIG = [
    {"out_channels": 16, "kernel_size": 8, "stride": 4},
    {"out_channels": 32, "kernel_size": 4, "stride": 2},
]
MLP_CONFIG = [
    32 * 9 * 9,  # Must match flattened output of CNN
    256,
    128,
    len(RIGHT_ONLY)
]
MODEL = NeuroevolutionNet(
    input_channels=1,
    num_actions=len(RIGHT_ONLY),
    cnn_config=CNN_CONFIG,
    mlp_config=MLP_CONFIG
)
ENV_NAME = 'SuperMarioBros-2-1-v0'
ACTION_SET = RIGHT_ONLY
DEVICE = "cpu"
DIRECTORY = "../../runs/neuroevolution/individual_level_training_1/2-1"
GENERATIONS = 300
POPULATION_SIZE = 100
MAX_STEPS_PER_EPISODE = 5000
MUTATION_RATE_RANGE = (0.14, 0.25)
MUTATION_STRENGTH_RANGE = (0.14, 0.19)
ROULETTE_WHEEL_SELECTION_TEMPERATURE = 0.98
ELITISM = 4
MUTATION_RATE_SIGMOID_DECAY = {"a": 0.05, "b": 0.058, "c": 100}
MUTATION_STRENGTH_SIGMOID_DECAY = {"a": 0.05, "b": 0.058, "c": 100}

trainer = NeuroevolutionTrainer(
        base_model=MODEL,
        env_name=ENV_NAME,
        action_set=ACTION_SET,
        device=DEVICE,
        directory=DIRECTORY,
        generations=GENERATIONS,
        population_size=POPULATION_SIZE,
        max_steps_per_episode=MAX_STEPS_PER_EPISODE,
        mutation_rate_range=MUTATION_RATE_RANGE,
        mutation_strength_range=MUTATION_STRENGTH_RANGE,
        roulette_wheel_selection_temperature=ROULETTE_WHEEL_SELECTION_TEMPERATURE,
        elitism=ELITISM,
        mutation_rate_sigmoid_decay=MUTATION_RATE_SIGMOID_DECAY,
        mutation_strength_sigmoid_decay=MUTATION_STRENGTH_SIGMOID_DECAY
    )

trainer.run()
trainer.record_best_agent()
trainer.save_metrics()
trainer.save_best_model()

# Individual Level Training 1 (Finden einer Lösung für 2-2) Level geschafft

Erster Versuch: max_fit_3163__avg_fit_1515__env_name_SuperMarioBros-2-2-v0__action_set_5
- Benutze dieselben Parameter wie in 1-2/max_fit_2441__avg_fit_531__env_name_SuperMarioBros-1-2-v0__action_set_5, da hier der höchste score in level 1-2 erzielt wurde. Ergebnis sehr gut, einfache simple aber effektive Taktik

In [ ]:
from src.non_reinforcement.neuroevolution.neuroevolution_trainer import NeuroevolutionTrainer
from src.non_reinforcement.neuroevolution.neuroevolution_net import NeuroevolutionNet
from gym_super_mario_bros.actions import RIGHT_ONLY
%config InlineBackend.figure_format = 'retina'

# Constants
CNN_CONFIG = [
    {"out_channels": 16, "kernel_size": 8, "stride": 4},
    {"out_channels": 32, "kernel_size": 4, "stride": 2},
]
MLP_CONFIG = [
    32 * 9 * 9,  # Must match flattened output of CNN
    256,
    128,
    len(RIGHT_ONLY)
]
MODEL = NeuroevolutionNet(
    input_channels=1,
    num_actions=len(RIGHT_ONLY),
    cnn_config=CNN_CONFIG,
    mlp_config=MLP_CONFIG
)
ENV_NAME = 'SuperMarioBros-2-2-v0'
ACTION_SET = RIGHT_ONLY
DEVICE = "cpu"
DIRECTORY = "../../runs/neuroevolution/individual_level_training_1/2-2"
GENERATIONS = 300
POPULATION_SIZE = 100
MAX_STEPS_PER_EPISODE = 5000
MUTATION_RATE_RANGE = (0.14, 0.25)
MUTATION_STRENGTH_RANGE = (0.14, 0.19)
ROULETTE_WHEEL_SELECTION_TEMPERATURE = 0.98
ELITISM = 4
MUTATION_RATE_SIGMOID_DECAY = {"a": 0.05, "b": 0.058, "c": 100}
MUTATION_STRENGTH_SIGMOID_DECAY = {"a": 0.05, "b": 0.058, "c": 100}

trainer = NeuroevolutionTrainer(
        base_model=MODEL,
        env_name=ENV_NAME,
        action_set=ACTION_SET,
        device=DEVICE,
        directory=DIRECTORY,
        generations=GENERATIONS,
        population_size=POPULATION_SIZE,
        max_steps_per_episode=MAX_STEPS_PER_EPISODE,
        mutation_rate_range=MUTATION_RATE_RANGE,
        mutation_strength_range=MUTATION_STRENGTH_RANGE,
        roulette_wheel_selection_temperature=ROULETTE_WHEEL_SELECTION_TEMPERATURE,
        elitism=ELITISM,
        mutation_rate_sigmoid_decay=MUTATION_RATE_SIGMOID_DECAY,
        mutation_strength_sigmoid_decay=MUTATION_STRENGTH_SIGMOID_DECAY
    )

trainer.run()
trainer.record_best_agent()
trainer.save_metrics()
trainer.save_best_model()

# Individual Level Training 1 (Finden einer Lösung für 2-3)

Erster Versuch: max_fit_3194__avg_fit_843__env_name_SuperMarioBros-2-3-v0__action_set_5
- Benutze dieselben Parameter wie in 1-2/max_fit_2441__avg_fit_531__env_name_SuperMarioBros-1-2-v0__action_set_5, da diese hyperparameter die besten Ergebnisse geliefert haben. Level fast geschafft.

In [ ]:
from src.non_reinforcement.neuroevolution.neuroevolution_trainer import NeuroevolutionTrainer
from src.non_reinforcement.neuroevolution.neuroevolution_net import NeuroevolutionNet
from gym_super_mario_bros.actions import RIGHT_ONLY
%config InlineBackend.figure_format = 'retina'

# Constants
CNN_CONFIG = [
    {"out_channels": 16, "kernel_size": 8, "stride": 4},
    {"out_channels": 32, "kernel_size": 4, "stride": 2},
]
MLP_CONFIG = [
    32 * 9 * 9,  # Must match flattened output of CNN
    256,
    128,
    len(RIGHT_ONLY)
]
MODEL = NeuroevolutionNet(
    input_channels=1,
    num_actions=len(RIGHT_ONLY),
    cnn_config=CNN_CONFIG,
    mlp_config=MLP_CONFIG
)
ENV_NAME = 'SuperMarioBros-2-3-v0'
ACTION_SET = RIGHT_ONLY
DEVICE = "cpu"
DIRECTORY = "../../runs/neuroevolution/individual_level_training_1/2-3"
GENERATIONS = 300
POPULATION_SIZE = 100
MAX_STEPS_PER_EPISODE = 5000
MUTATION_RATE_RANGE = (0.14, 0.25)
MUTATION_STRENGTH_RANGE = (0.14, 0.19)
ROULETTE_WHEEL_SELECTION_TEMPERATURE = 0.98
ELITISM = 4
MUTATION_RATE_SIGMOID_DECAY = {"a": 0.05, "b": 0.058, "c": 100}
MUTATION_STRENGTH_SIGMOID_DECAY = {"a": 0.05, "b": 0.058, "c": 100}

trainer = NeuroevolutionTrainer(
        base_model=MODEL,
        env_name=ENV_NAME,
        action_set=ACTION_SET,
        device=DEVICE,
        directory=DIRECTORY,
        generations=GENERATIONS,
        population_size=POPULATION_SIZE,
        max_steps_per_episode=MAX_STEPS_PER_EPISODE,
        mutation_rate_range=MUTATION_RATE_RANGE,
        mutation_strength_range=MUTATION_STRENGTH_RANGE,
        roulette_wheel_selection_temperature=ROULETTE_WHEEL_SELECTION_TEMPERATURE,
        elitism=ELITISM,
        mutation_rate_sigmoid_decay=MUTATION_RATE_SIGMOID_DECAY,
        mutation_strength_sigmoid_decay=MUTATION_STRENGTH_SIGMOID_DECAY
    )

trainer.run()
trainer.record_best_agent()
trainer.save_metrics()
trainer.save_best_model()